# Python Technical Challenge

## _Searching an algorithm that sorts words from left to right line by line_

c: Dominic Catana <catanadominic@gmail.com>

In [71]:
# -----------------ONLY METHODS AND IMPORTS------------------------
import json
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from operator import itemgetter

def extract_points(ocr_dataset):
    list_of_coords = []
    for it in ocr_dataset:
        for y in it['points']:
            list_of_coords.append(y)
    return list_of_coords

def get_ocr_data(path):
    f = open(path,)
    data = json.load(f)['ocr'];
    return data

def change_points_to_coords(ocr_dataset):
    l = []
    c = 0;
    x = -1;
    y = -1;
    min = 100000
    max = 0
    for c in ocr_dataset:
        x = c["points"][3][0]
        y = c["points"][3][1]
        w = c["points"][1][0]-c["points"][0][0]
        h = c["points"][2][1]-c["points"][1][1]
        c["points"]=[x,y,w,h]
    return ocr_dataset

def useful_data(coords):
    values = []
    original_list = []
    for i in coords:
        values.append(i['text'])
        values.append(i['points'][0])
        values.append(i['points'][2])   # take only the text and the opposite corners (left buttom, right top)
        original_list.append(values)
        values = []
    return original_list

def get_coords_sum(coords):
    values = []
    original_list = []
    for i in coords:
        values.append(i['text'])
        values.append(i['points'][0])
        values.append(i['points'][0][0]+i['points'][0][1])   # take text, sum of coordinates first point and top right corner's y
        values.append(i['points'][2][1])                   
        original_list.append(values)
        values = []
    return original_list 
    

def sorting_algorithm(coords):
    initial_coords = useful_data(coords)
    sum_coords = get_coords_sum(coords)
    return sum_coords
    



In [72]:
# --------------__main__--------------------

data = get_ocr_data("datasets/menu2.json")
#print(data)

print(sorting_algorithm(data))












[['BROILING', [133.87486267089844, 511.69140625], 645.5662689208984, 520.0508422851562], ['OUR', [110.84564971923828, 510.99517822265625], 621.8408279418945, 521.0720825195312], ['ALL', [89.71671295166016, 510.99517822265625], 600.7118911743164, 521.0720825195312], ['DONE', [191.95838928222656, 510.49652099609375], 702.4549102783203, 519.6204833984375], ['IS', [179.46607767535104, 510.31321618621934], 689.7792938615704, 521.1172380769398], ['OVER', [221.04116821289062, 509.0448303222656], 730.0859985351562, 518.796630859375], ['LIVE', [247.25387573242188, 508.1828308105469], 755.4367065429688, 517.3831176757812], ['CHARCOAL', [270.950927734375, 508.54278564453125], 779.4937133789062, 515.0728759765625], ['Coffee', [139.4509735107422, 490.1913146972656], 629.6422882080078, 500.5932312011719], ['Tea', [200.88742065429688, 489.5411682128906], 690.4285888671875, 499.943115234375], ['Milk', [252.44467163085938, 488.0399169921875], 740.4845886230469, 496.89361572265625], ['Peach', [178.78329